In [19]:
import numpy as np
import cv2

In [20]:
src=cv2.imread("lena.bmp",cv2.IMREAD_GRAYSCALE)
#hw2實做過的function
rows,cols=src.shape
srcBinary=np.zeros(shape=src.shape,dtype=src.dtype)
for i in range (rows):
    for j in range(cols):
        if src[i,j]>128:
            srcBinary[i,j]=255
        else:
            srcBinary[i,j]=0
cv2.imwrite("lenaBinary.png",srcBinary)

True

In [38]:
def downsampling (image,sampleFactor):
    """
    input:image,Mat(opencv)
    sampleFactor: 要縮小的倍數
    output:image after downsample
    """
    rows,cols=image.shape
    rowsD=int(rows/sampleFactor)
    colsD=int(cols/sampleFactor)
    downsampleImg=np.zeros(shape=(rowsD,colsD),dtype=image.dtype)
    for i in range(rowsD):
        for j in range(colsD):
            downsampleImg[i,j]=image[i*sampleFactor,j*sampleFactor]
    return downsampleImg

In [39]:
downsample=downsampling(srcBinary,8)

In [5]:
def fYokoi(a1,a2,a3,a4):
    """
    input: str
    output: str
    """
    if ([a1,a2,a3,a4].count('r')==4):
        return 5
    else:
        return [a1,a2,a3,a4].count('q')

In [6]:
def hYokoi(b,c,d,e):
    """
    input: int
    output: str
    """
    if b==c and (d!=b or e!=b):
        return'q'
    elif b==c and (d==b and e==b):
        return'r'
    elif b!=c:
        return's'

In [7]:
def getNeighborPixels (image,position):
    """
    input: image,position
    output: np.array
    """
    positionX,positionY=position
    neighborhoodPixels=np.zeros(9)
    for i in range(3):
        for j in range(3):
            dstX=positionX+(i-1)
            dstY=positionY+(j-1)
            if((0<=dstX<image[0].size) and (0<=dstY<image[1].size)):
                neighborhoodPixels[3*j+i]=image[dstX,dstY]
            else:
                neighborhoodPixels[3*j+i]=0
    neighborhoodPixels=[neighborhoodPixels[4],neighborhoodPixels[5],neighborhoodPixels[1]
                       ,neighborhoodPixels[3],neighborhoodPixels[7],neighborhoodPixels[8]
                       ,neighborhoodPixels[2],neighborhoodPixels[0],neighborhoodPixels[6]]
    return neighborhoodPixels

In [8]:
def yokoiConnectivityNumber(image):
    """
    input: downsampled image
    output: chararray
    """
    rows,cols=image.shape
    connectNum = np.chararray(shape=image.shape,unicode=True)
    for i in range(rows):
        for j in range(cols):
            if(image[i,j]!=0):
                neighborhoodPixels= getNeighborPixels(image,(i,j))
                a1= hYokoi(neighborhoodPixels[0],neighborhoodPixels[1],neighborhoodPixels[6],neighborhoodPixels[2])
                a2= hYokoi(neighborhoodPixels[0],neighborhoodPixels[2],neighborhoodPixels[7],neighborhoodPixels[3])
                a3= hYokoi(neighborhoodPixels[0],neighborhoodPixels[3],neighborhoodPixels[8],neighborhoodPixels[4])
                a4= hYokoi(neighborhoodPixels[0],neighborhoodPixels[4],neighborhoodPixels[5],neighborhoodPixels[1])
                connectNum[i,j]=fYokoi(a1,a2,a3,a4)
            else:
                connectNum[i,j]=' '
            if connectNum[i,j]=='0':
                connectNum[i,j]=' '
    return connectNum

In [46]:
def yokoi(image):
    yokoiConnect = np.zeros(shape=image.shape,dtype=image.dtype)
    rows,cols=image.shape
    for i in range(rows):
        for j in range(cols):
            if image[i,j]!=0:
                a1 = a2 = a3 = a4 = "s"
                b = image[i,j]
                #a1
                c = d = e = 0
                if j != cols - 1 and i != 0:
                    c = image[i,j+1]
                    d = image[i-1,j+1]
                    e = image[i-1,j]
                elif i != 0:
                    e = image[i-1,j]
                elif j != cols - 1:
                    c = image[i,j+1]
                a1 = hYokoi(b, c, d, e)
                #a2
                c = d = e = 0
                if j != 0 and i != 0:
                    c = image[i-1,j]
                    d = image[i-1,j-1]
                    e = image[i,j-1]
                elif i != 0:
                    c = image[i-1,j]
                elif j != 0:
                    e = image[i,j-1]  
                a2 = hYokoi(b, c, d, e)
                #a3
                c = d = e = 0
                if j != 0 and i != rows - 1:
                    c = image[i,j-1]
                    d = image[i+1,j-1]
                    e = image[i+1,j]
                elif i != rows - 1:
                    e = image[i+1,j]
                elif j != 0:
                    c = image[i,j-1]  
                a3 = hYokoi(b, c, d, e)
                #a4
                c = d = e = 0
                if j != cols - 1 and i != rows - 1:
                    c = image[i+1,j]
                    d = image[i+1,j+1]
                    e = image[i,j+1]
                elif i != rows - 1:
                    c = image[i+1,j]
                elif j != cols - 1:
                    e = image[i,j+1]  
                a4 = hYokoi(b, c, d, e)
                yokoiConnect[i,j] = fYokoi(a1, a2, a3, a4)
    return yokoiConnect

def pairRelationshipOperator(YokoiConnectNum):
    """
    input:chararray
    output:chararray
    """
    markedImg= np.chararray(shape=YokoiConnectNum.shape,unicode=True)
    np.pad(YokoiConnectNum,((1,1),(1,1)), 'constant')
    rows,cols=YokoiConnectNum.shape
    for i in range(rows):
        for j in range(cols):
            if YokoiConnectNum[i,j]=='0':
                YokoiConnectNum[i,j]=' '
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            #neighborPixel=getNeighborPixels(YokoiConnectNum,(i,j))
            if(YokoiConnectNum[i,j]==''):
                markedImg[i,j]=' '
            else:
                x1=YokoiConnectNum[i+1,j]
                x2=YokoiConnectNum[i,j-1]
                x3=YokoiConnectNum[i-1,j]
                x4=YokoiConnectNum[i,j+1]
                if (([x1,x2,x3,x4].count('1')>=1) and YokoiConnectNum[i,j]=='1'):
                    markedImg[i,j]='p'
                else:
                    markedImg[i,j]='q'      
    return markedImg

In [40]:
def pairRelationshipOperator(image):
    yokoiConnect=yokoi(image)
    rows,cols=yokoiConnect.shape
    markedImg=np.zeros([rows,cols],str)
    for i in range(rows):
        for j in range(cols):
            if yokoiConnect[i,j]==1:
                if yokoiConnect[i-1,j]==1 and i>0:
                    markedImg[i,j]='p'
                elif i<(rows-1) and yokoiConnect[i+1,j]==1:
                    markedImg[i,j]='p'
                elif j<(cols-1) and yokoiConnect[i,j+1]==1:
                    markedImg[i,j]='p'
                elif yokoiConnect[i,j-1]==1 and j>0:
                    markedImg[i,j]='p'
                else:
                    markedImg[i,j]='q'
            else:
                markedImg[i,j]='q'
    return markedImg

In [28]:
def connectedShrinkOperator(image):
    yokoiConnect = yokoi(image)
    markedImg=pairRelationshipOperator(image)
    rows,cols=markedImg.shape
    for i in range(rows):
        for j in range(cols):
            if markedImg[i,j] == 'p' and yokoiConnect[i,j]==1:
                image[i,j] = 0 
                yokoiConnect=yokoi(image)
    return image     

In [48]:
thinned=downsample
i=0

while True:

    yokoiConnect=yokoi(thinned)
    markedImg = pairRelationshipOperator(yokoiConnect)
    temp = connectedShrinkOperator(thinned)
    if (temp.all()==thinned.all() ):
        break    
    thinned = temp
    i = i + 1
cv2.imwrite('result.png', thinned)

True